<h1 style="color: white; background: #ADD8E6; text-align:center; font-size: 2.3em;"> MoA + Feature Engeneering + Keras </h1>

## File Paths

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/README.md
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.py
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/test_ml_stratifiers.py
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/__init__.py
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/ml_stratifiers.py
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/__init__.py
/kaggle/input/pytorchtabnet/pytorch_tabnet-1.2.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
/

# 1. Preparation

In [2]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


## 1.1 Imports

In [3]:
import numpy as np
import pandas as pd

## Keras and Tf ##

import tensorflow as tf
import tensorflow.keras.callbacks as C
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.optimizers as O
import tensorflow.keras.utils as U
import tensorflow.keras.backend as K

#Utils

import random
import os
import math
import tqdm

## Preprocessing and sklearn ##

from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
sys.path.append('../input/rank-gauss')
from gauss_rank_scaler import GaussRankScaler

## Pytorch ##

import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.optimizer import Optimizer

## TabNet
sys.path.append('../input/pytorch-tabnet')
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


## 1.2 Reading Files

In [4]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sub = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [5]:
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [6]:
target_cols = sub.columns[1:]

## 1.3 Set Seed for Reproducibility

In [7]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

# 2. Feature Engineering/Selection

## 2.1 Remove Control Group

In [8]:
# Remove control group
train_features = train_features[train_features["cp_type"] != "ctl_vehicle"]
test_features = test_features[test_features["cp_type"] != "ctl_vehicle"]
train_targets = train_targets.iloc[train_features.index]
train_features.reset_index(drop = True, inplace = True)
test_features.reset_index(drop = True, inplace = True)
train_targets.reset_index(drop = True, inplace = True)

## 2.2 Variance Threshold

In [9]:
# #Feature Selection with Variance Threshold
# #Had problems with submission on this
c_n = [f for f in list(train_features.columns) if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
mask = (train_features[c_n].var() >= 0.7).values
tmp = train_features[c_n].loc[:, mask]
train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
tmp = test_features[c_n].loc[:, mask]
test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)

## 2.3 PCA

In [10]:
# We are actully adding the feature from the pca into the original data in here, it helps the score for some reason
def pca(train, test, n_comp = 55, type_ = "", SEED = 123):
    
    kind = "g" if type_ == GENES else "c"
    data = pd.concat([train[type_], test[type_]])
#     train.drop(type_, axis = 1, inplace = True)
#     test.drop(type_, axis = 1, inplace = True)
    pca = PCA(n_components= n_comp, random_state = SEED)
    data = pd.DataFrame(pca.fit_transform(data), columns = [f'pca-{kind}-{i + 1}' for i in range(n_comp)])
    train_ = data.iloc[:train.shape[0]]
    test_ = data.iloc[train.shape[0]:].reset_index(drop = True)
    train = pd.concat([train, train_], axis = 1)
    test = pd.concat([test, test_], axis = 1)
    return train, test

In [11]:
#All the col that has genes and cells
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [12]:
train_features, test_features = pca(train_features, test_features, n_comp = 120, type_ = GENES)
train_features, test_features = pca(train_features, test_features, n_comp = 20, type_ = CELLS)

In [13]:
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-2,g-3,g-4,g-5,g-6,...,pca-c-11,pca-c-12,pca-c-13,pca-c-14,pca-c-15,pca-c-16,pca-c-17,pca-c-18,pca-c-19,pca-c-20
0,id_000644bb2,trt_cp,24,D1,1.0620,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.687773,-0.551501,-0.576611,-0.081799,0.527897,-0.889118,-1.019991,-0.293436,-0.827717,-0.569132
1,id_000779bfc,trt_cp,72,D1,0.0743,0.2991,0.0604,1.0190,0.5207,0.2341,...,0.820746,-0.173563,0.728606,0.250554,1.205988,-0.536938,0.840609,-0.021688,0.218566,0.292075
2,id_000a6266a,trt_cp,48,D1,0.6280,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.217845,0.178157,0.415141,0.494522,-0.435289,0.893302,-0.328090,1.046894,0.082787,0.789685
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-0.039129,0.144012,0.878549,-1.351130,1.604516,0.233892,0.639763,-0.375438,1.440603,1.366497
4,id_001626bd3,trt_cp,72,D2,-0.3254,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,-0.105938,-0.255500,0.175404,-0.016431,-0.772789,0.425729,0.391218,0.121719,0.660142,-0.449516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,trt_cp,72,D1,0.1608,0.2551,-0.2239,-0.2431,0.4256,-0.1166,...,-0.860570,0.779536,-0.072007,0.329810,-0.058445,0.725834,-0.019420,0.898136,0.302271,-0.174918
21944,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,-0.305824,0.780934,-0.242487,-0.132851,0.948908,0.631465,0.698903,-0.791345,0.653110,0.196764
21945,id_fffb70c0c,trt_cp,24,D2,-1.3260,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0.245729,0.718856,-0.561454,0.453753,0.168263,0.059555,0.487017,-0.254938,0.079887,-0.622716
21946,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.4392,0.2044,0.8531,-0.0343,0.0323,...,0.808229,-1.690100,-0.309677,-0.166884,0.761904,-0.164638,-0.464094,0.808345,-0.334289,-0.019326


In [14]:
#All the col that has genes and cells
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
cols_numeric = [feat for feat in list(train_features.columns) if feat not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]

In [15]:
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-2,g-3,g-4,g-5,g-6,...,pca-c-11,pca-c-12,pca-c-13,pca-c-14,pca-c-15,pca-c-16,pca-c-17,pca-c-18,pca-c-19,pca-c-20
0,id_000644bb2,trt_cp,24,D1,1.0620,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.687773,-0.551501,-0.576611,-0.081799,0.527897,-0.889118,-1.019991,-0.293436,-0.827717,-0.569132
1,id_000779bfc,trt_cp,72,D1,0.0743,0.2991,0.0604,1.0190,0.5207,0.2341,...,0.820746,-0.173563,0.728606,0.250554,1.205988,-0.536938,0.840609,-0.021688,0.218566,0.292075
2,id_000a6266a,trt_cp,48,D1,0.6280,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.217845,0.178157,0.415141,0.494522,-0.435289,0.893302,-0.328090,1.046894,0.082787,0.789685
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-0.039129,0.144012,0.878549,-1.351130,1.604516,0.233892,0.639763,-0.375438,1.440603,1.366497
4,id_001626bd3,trt_cp,72,D2,-0.3254,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,-0.105938,-0.255500,0.175404,-0.016431,-0.772789,0.425729,0.391218,0.121719,0.660142,-0.449516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,trt_cp,72,D1,0.1608,0.2551,-0.2239,-0.2431,0.4256,-0.1166,...,-0.860570,0.779536,-0.072007,0.329810,-0.058445,0.725834,-0.019420,0.898136,0.302271,-0.174918
21944,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,-0.305824,0.780934,-0.242487,-0.132851,0.948908,0.631465,0.698903,-0.791345,0.653110,0.196764
21945,id_fffb70c0c,trt_cp,24,D2,-1.3260,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0.245729,0.718856,-0.561454,0.453753,0.168263,0.059555,0.487017,-0.254938,0.079887,-0.622716
21946,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.4392,0.2044,0.8531,-0.0343,0.0323,...,0.808229,-1.690100,-0.309677,-0.166884,0.761904,-0.164638,-0.464094,0.808345,-0.334289,-0.019326


## 2.4 Feature Creation using KMeans Clustering

In [16]:
def create_cluster(train, test, kind, n_cluster, str_kind):
    
    data = pd.concat([train[kind], test[kind]], ignore_index = True)
    kmeans = KMeans(n_clusters = n_cluster, random_state = 0,).fit(data)
    train[f'clusters_{str_kind}'] = kmeans.labels_[:train.shape[0]]
    test[f'clusters_{str_kind}'] = kmeans.labels_[train.shape[0]:]
    train = pd.get_dummies(train, columns = [f'clusters_{str_kind}'])
    test = pd.get_dummies(test, columns = [f'clusters_{str_kind}'])
    
    return train, test

In [17]:
# train_features, test_features = create_cluster(train_features, test_features, GENES, 35, "g")
# train_features, test_features = create_cluster(train_features, test_features, CELLS, 5, "c")

In [18]:
data = pd.concat([train_features, test_features], ignore_index = True)

## 2.5 Gauss Rank Scaler

In [19]:
#Normalize the data
scaler = GaussRankScaler()
data[cols_numeric]  = scaler.fit_transform(data[cols_numeric])

## 2.6 Stats of Features

In [20]:
for stats in tqdm.tqdm(["mean", "std", "kurt", "skew"]):
    data["g_" + stats] = getattr(data[GENES], stats)(axis = 1)
    data["c_" + stats] = getattr(data[CELLS], stats)(axis = 1)    
    data["gc_" + stats] = getattr(data[GENES + CELLS], stats)(axis = 1)

100%|██████████| 4/4 [00:04<00:00,  1.02s/it]


In [21]:
#Spliting the data back into train and test
train_features = data.iloc[:train_features.shape[0]].reset_index(drop = True)
test_features = data.iloc[train_features.shape[0]:].reset_index(drop = True)

# 3. Preprocessing

## 3.1 Get Dummies & Dropping sig_id

In [22]:
def preprocess(data, target=False):
    data.drop("sig_id", axis=1, inplace=True)
    if target == False:
        data["cp_dose_time"] = data["cp_dose"] + "_" + data["cp_time"].astype("str")
        data["cp_dose_time"] = data["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})
    return data

In [23]:
train_features = preprocess(train_features)
train_targets = preprocess(train_targets, target=True)
train_features = pd.get_dummies(train_features)
test_features = preprocess(test_features)
test_features = pd.get_dummies(test_features)

In [24]:
train_features

,cp_time,g-0,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,g_kurt,c_kurt,gc_kurt,g_skew,c_skew,gc_skew,cp_dose_time,cp_type_trt_cp,cp_dose_D1,cp_dose_D2
0,24,0.731433,-0.254104,-0.615315,-0.194236,-0.772257,-1.061067,0.005623,0.513989,-0.130963,...,-0.006219,-0.250230,-0.026184,-0.008259,0.118667,-0.036543,0,1,1,0
1,72,0.017020,0.213820,0.027107,0.849834,0.482972,0.217779,0.363904,-0.298420,0.730029,...,-0.053247,0.111352,-0.099643,0.096482,-0.043717,0.003754,2,1,1,0
2,48,0.462792,0.981712,-0.091777,-0.039121,1.055526,0.161005,0.242395,0.050070,0.975415,...,-0.362998,-0.051865,-0.282383,-0.017658,0.379548,0.010043,1,1,1,0
3,48,-0.531916,-0.269127,0.455184,1.595325,-0.621171,-1.509230,0.206249,-0.066407,-0.889590,...,-0.880856,4.444686,-0.909214,0.108938,2.010409,0.273694,1,1,1,0
4,72,-0.341668,0.808567,0.604615,0.951946,-0.630950,-0.243933,-0.103294,-0.684346,0.741863,...,-0.351803,-0.477550,-0.240858,-0.130955,0.191181,-0.201224,5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,72,0.089366,0.176202,-0.226762,-0.244678,0.396468,-0.094758,-0.130161,-0.582338,0.056587,...,-0.031021,-0.228406,-0.011154,-0.083142,0.473883,-0.017303,2,1,1,0
21944,24,0.072056,-0.131998,-0.499092,-0.479222,0.667644,0.530397,0.333563,-0.356318,0.048109,...,0.196997,-0.145076,0.253493,-0.222104,0.367298,-0.168144,3,1,0,1
21945,24,-1.346695,-0.369847,0.878667,-0.735139,0.613056,-0.194656,-0.446960,0.479298,0.549999,...,0.102816,-0.157828,0.107293,-0.021382,0.484219,0.055402,3,1,0,1
21946,24,0.490027,0.329593,0.157679,0.732402,0.001062,0.034881,0.079245,0.405150,-0.742506,...,-1.023389,-0.579405,-0.914582,-0.140780,0.200423,-0.189765,0,1,1,0


## 3.2 Define Error/Loss Metric and Prediction clipping

In [25]:
def multi_log_loss(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
    return np.mean(losses)

In [26]:
def clip(y_pred, c_min, c_max):
    return np.clip(y_pred, c_min, c_max)

In [27]:
p_min = 0.001
p_max = 0.999

# 4. Create Model and Perform KFold

## 4.1 Two Layer DNN

In [28]:
#The model param is tuned with optuna 
#The leaky ReLu performs the best
def create_model_v1(shape):
    model = M.Sequential()
    model.add(L.Input(shape = (shape)))
    model.add(L.BatchNormalization())
    model.add(L.Dropout(0.397816152960733))
    model.add(tfa.layers.WeightNormalization(L.Dense(521)))
    model.add(L.LeakyReLU(alpha = 0.10059420295821832))
    model.add(L.BatchNormalization())
    model.add(L.Dropout(0.4365714774136811))
    model.add(tfa.layers.WeightNormalization(L.Dense(1634)))
    model.add(L.LeakyReLU(alpha = 0.10059420295821832))
    model.add(L.BatchNormalization())
    model.add(L.Dropout(0.4365714774136811))
    model.add(tfa.layers.WeightNormalization(L.Dense(206, activation = "sigmoid")))
    model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756), 
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0015),
                  metrics=tf.keras.metrics.BinaryCrossentropy())
    return model

## 4.2 TabNet

### 4.2.1 AdaBelief Optimizer

In [29]:
version_higher = ( torch.__version__ >= "1.5.0" )

class AdaBelief(Optimizer):
    r"""Implements AdaBelief algorithm. Modified from Adam in PyTorch
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        amsgrad (boolean, optional): whether to use the AMSGrad variant of this
            algorithm from the paper `On the Convergence of Adam and Beyond`_
            (default: False)
        weight_decouple (boolean, optional): ( default: False) If set as True, then
            the optimizer uses decoupled weight decay as in AdamW
        fixed_decay (boolean, optional): (default: False) This is used when weight_decouple
            is set as True.
            When fixed_decay == True, the weight decay is performed as
            $W_{new} = W_{old} - W_{old} \times decay$.
            When fixed_decay == False, the weight decay is performed as
            $W_{new} = W_{old} - W_{old} \times decay \times lr$. Note that in this case, the
            weight decay ratio decreases with learning rate (lr).
        rectify (boolean, optional): (default: False) If set as True, then perform the rectified
            update similar to RAdam
    reference: AdaBelief Optimizer, adapting stepsizes by the belief in observed gradients
               NeurIPS 2020 Spotlight
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False, weight_decouple = False, fixed_decay=False, rectify = False ):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad)
        super(AdaBelief, self).__init__(params, defaults)

        self.weight_decouple = weight_decouple
        self.rectify = rectify
        self.fixed_decay = fixed_decay
        if self.weight_decouple:
            print('Weight decoupling enabled in AdaBelief')
            if self.fixed_decay:
                print('Weight decay fixed')
        if self.rectify:
            print('Rectification enabled in AdaBelief')
        if amsgrad:
            print('AMS enabled in AdaBelief')
    def __setstate__(self, state):
        super(AdaBelief, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    def reset(self):
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                amsgrad = group['amsgrad']

                # State initialization
                state['step'] = 0
                # Exponential moving average of gradient values
                state['exp_avg'] = torch.zeros_like(p.data,
                                   memory_format=torch.preserve_format) if version_higher else torch.zeros_like(p.data)

                # Exponential moving average of squared gradient values
                state['exp_avg_var'] = torch.zeros_like(p.data,
                                    memory_format=torch.preserve_format) if version_higher else torch.zeros_like(p.data)
                if amsgrad:
                    # Maintains max of all exp. moving avg. of sq. grad. values
                    state['max_exp_avg_var'] = torch.zeros_like(p.data,
                                    memory_format=torch.preserve_format) if version_higher else torch.zeros_like(p.data)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('AdaBelief does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']

                state = self.state[p]
               
                beta1, beta2 = group['betas']

                # State initialization
                if len(state) == 0:
                    state['rho_inf'] = 2.0 / (1.0 - beta2) - 1.0
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data,
                                    memory_format=torch.preserve_format) if version_higher else torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_var'] = torch.zeros_like(p.data,
                                    memory_format=torch.preserve_format) if version_higher else torch.zeros_like(p.data)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_var'] = torch.zeros_like(p.data,
                                    memory_format=torch.preserve_format) if version_higher else torch.zeros_like(p.data)

                # get current state variable
                exp_avg, exp_avg_var = state['exp_avg'], state['exp_avg_var']

                state['step'] += 1
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']

                # perform weight decay, check if decoupled weight decay
                if self.weight_decouple:
                    if not self.fixed_decay:
                        p.data.mul_(1.0 - group['lr'] * group['weight_decay'])
                    else:
                        p.data.mul_(1.0 - group['weight_decay'])
                else:
                    if group['weight_decay'] != 0:
                        grad.add_(group['weight_decay'], p.data)

                # Update first and second moment running average
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                grad_residual = grad - exp_avg
                exp_avg_var.mul_(beta2).addcmul_(1 - beta2, grad_residual, grad_residual)

                if amsgrad:
                    max_exp_avg_var = state['max_exp_avg_var']
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_var, exp_avg_var, out=max_exp_avg_var)

                    # Use the max. for normalizing running avg. of gradient
                    denom = (max_exp_avg_var.add_(group['eps']).sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
                else:
                    denom = (exp_avg_var.add_(group['eps']).sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])

                if not self.rectify:
                    # Default update
                    step_size = group['lr'] / bias_correction1
                    p.data.addcdiv_(-step_size, exp_avg, denom)

                else:# Rectified update
                    # calculate rho_t
                    state['rho_t'] = state['rho_inf'] - 2 * state['step'] * beta2 ** state['step'] / (
                            1.0 - beta2 ** state['step'])

                    if state['rho_t'] > 4: # perform Adam style update if variance is small
                        rho_inf, rho_t = state['rho_inf'], state['rho_t']
                        rt = (rho_t - 4.0) * (rho_t - 2.0) * rho_inf / (rho_inf - 4.0) / (rho_inf - 2.0) / rho_t
                        rt = math.sqrt(rt)

                        step_size = rt * group['lr'] / bias_correction1

                        p.data.addcdiv_(-step_size, exp_avg, denom)

                    else: # perform SGD style update
                        p.data.add_( -group['lr'], exp_avg)

        return loss

### 4.2.2 Model Params

In [30]:
MAX_EPOCH = 200
tabnet_params = dict(
    n_d = 28,
    n_a = 28,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = AdaBelief,
    optimizer_params = dict(lr = 7e-3, weight_decay = 1e-5, eps=1e-15),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 7e-6, factor = 0.4),
    scheduler_fn = ReduceLROnPlateau,
    seed = 1,
    verbose = 10
)

### 4.2.3 Custom Metric for TabNet

In [31]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

## 4.3 KFold

In [32]:
def make_folds(folds):
    models = []
    predictions = []
    
    for i in range(3):
        
        seed = i
        seed_everything(seed)
        
        model_list = list()
        kfold = MultilabelStratifiedKFold(folds, shuffle = True, random_state = seed)
        oof_preds = train_targets.copy()
        
        print(f"REAPEAT NUMBER {i + 1} SEED {seed}")
        print("TabNet")
        
        for fold, (train_ind, val_ind) in enumerate(kfold.split(train_targets, train_targets)):
            
            if fold % 2 == 0:
                model = TabNetRegressor(**tabnet_params)

                print('\n')
                print('-'*50)
                print(f'Training fold {fold + 1}')
                
                
                model.fit(X_train = train_features.values[train_ind], y_train = train_targets.values[train_ind], eval_set = [(train_features.values[val_ind], train_targets.values[val_ind])], 
                          eval_name = ["val"], eval_metric = ["logits_ll"], max_epochs = MAX_EPOCH,
                        patience = 20, batch_size = 256, virtual_batch_size = 32, num_workers = 2, drop_last = False,
                        # To use binary cross entropy because this is not a regression problem
                        loss_fn = F.binary_cross_entropy_with_logits
                        )
                model_list.append(model)
                preds = model.predict(train_features.values[val_ind])
                preds = 1 / (1 + np.exp(-preds))
                oof_preds.loc[val_ind, :] = clip(preds, p_min, p_max)
            
            else:
                model = create_model_v1(996)
            
                print('\n')
                print('-'*50)
                print(f'Training fold {fold + 1}')

                checkpoint_path = f'repeat:{seed}_Fold:{fold}.hdf5'
                cb_checkpt = C.ModelCheckpoint(checkpoint_path, monitor = 'val_binary_crossentropy', verbose = 2, save_best_only = True, save_weights_only = True, mode = 'auto')
                reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_binary_crossentropy',
                                                             mode = 'min',
                                                             factor = 0.2,
                                                             patience = 3,
                                                             verbose = 3)
                early = tf.keras.callbacks.EarlyStopping(monitor = 'val_binary_crossentropy',
                                                              mode = 'min',
                                                              patience = 10,
                                                              restore_best_weights = True,
                                                              verbose = 3)
                model.fit(train_features.values[train_ind],
                    train_targets.values[train_ind], validation_data=(train_features.values[val_ind], train_targets.values[val_ind]),
                        epochs=100, batch_size=128, verbose=2, callbacks = [reduce_lr, early, cb_checkpt])
                model_list.append(model)
                model.load_weights(checkpoint_path)
                oof_preds.loc[val_ind, :] = clip(model.predict(train_features.values[val_ind]), p_min, p_max)

        m, oof = model_list, oof_preds
        models = models + m
        
        predictions.append(oof)
    return models, predictions

In [33]:
models, oof_preds = make_folds(10)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


REAPEAT NUMBER 1 SEED 0
TabNet
Device used : cuda


--------------------------------------------------
Training fold 1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:135: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)


epoch 0  | loss: 0.1785  | val_logits_ll: 0.02261 |  0:00:03s
epoch 10 | loss: 0.01798 | val_logits_ll: 0.01775 |  0:00:32s
epoch 20 | loss: 0.0171  | val_logits_ll: 0.01709 |  0:01:03s
epoch 30 | loss: 0.01684 | val_logits_ll: 0.01722 |  0:01:33s
epoch 40 | loss: 0.01581 | val_logits_ll: 0.01669 |  0:02:04s
epoch 50 | loss: 0.01415 | val_logits_ll: 0.01685 |  0:02:32s
epoch 60 | loss: 0.01323 | val_logits_ll: 0.01739 |  0:03:03s

Early stopping occured at epoch 62 with best_epoch = 42 and best_val_logits_ll = 0.0165
Best weights from best epoch are automatically used!


--------------------------------------------------
Training fold 2
Epoch 1/100

Epoch 00001: val_binary_crossentropy improved from inf to 0.04444, saving model to repeat:0_Fold:1.hdf5
155/155 - 2s - loss: 0.3495 - binary_crossentropy: 0.3484 - val_loss: 0.0471 - val_binary_crossentropy: 0.0444
Epoch 2/100

Epoch 00002: val_binary_crossentropy improved from 0.04444 to 0.02263, saving model to repeat:0_Fold:1.hdf5
155/15

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


REAPEAT NUMBER 2 SEED 1
TabNet
Device used : cuda


--------------------------------------------------
Training fold 1
epoch 0  | loss: 0.17945 | val_logits_ll: 0.02286 |  0:00:03s
epoch 10 | loss: 0.01853 | val_logits_ll: 0.01831 |  0:00:35s
epoch 20 | loss: 0.01724 | val_logits_ll: 0.01727 |  0:01:09s
epoch 30 | loss: 0.01694 | val_logits_ll: 0.01703 |  0:01:42s
epoch 40 | loss: 0.01668 | val_logits_ll: 0.01674 |  0:02:15s
epoch 50 | loss: 0.01562 | val_logits_ll: 0.01642 |  0:02:50s
epoch 60 | loss: 0.01416 | val_logits_ll: 0.01662 |  0:03:23s

Early stopping occured at epoch 67 with best_epoch = 47 and best_val_logits_ll = 0.01642
Best weights from best epoch are automatically used!


--------------------------------------------------
Training fold 2
Epoch 1/100

Epoch 00001: val_binary_crossentropy improved from inf to 0.03970, saving model to repeat:1_Fold:1.hdf5
155/155 - 2s - loss: 0.3519 - binary_crossentropy: 0.3508 - val_loss: 0.0424 - val_binary_crossentropy: 0.0397
Epoch 2

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


REAPEAT NUMBER 3 SEED 2
TabNet
Device used : cuda


--------------------------------------------------
Training fold 1
epoch 0  | loss: 0.17833 | val_logits_ll: 0.02244 |  0:00:04s
epoch 10 | loss: 0.01806 | val_logits_ll: 0.01757 |  0:00:38s
epoch 20 | loss: 0.01723 | val_logits_ll: 0.01715 |  0:01:13s
epoch 30 | loss: 0.01691 | val_logits_ll: 0.01695 |  0:01:45s
epoch 40 | loss: 0.01675 | val_logits_ll: 0.01684 |  0:02:20s
epoch 50 | loss: 0.01654 | val_logits_ll: 0.01691 |  0:02:53s
epoch 60 | loss: 0.01539 | val_logits_ll: 0.0164  |  0:03:27s
epoch 70 | loss: 0.01405 | val_logits_ll: 0.01647 |  0:04:00s

Early stopping occured at epoch 77 with best_epoch = 57 and best_val_logits_ll = 0.01617
Best weights from best epoch are automatically used!


--------------------------------------------------
Training fold 2
Epoch 1/100

Epoch 00001: val_binary_crossentropy improved from inf to 0.04181, saving model to repeat:2_Fold:1.hdf5
155/155 - 2s - loss: 0.3507 - binary_crossentropy: 0.349

In [34]:
mean_oof_preds = train_targets.copy()
mean_oof_preds.loc[:, target_cols] = 0
scores = []
for i, p in enumerate(oof_preds):
    loss = multi_log_loss(train_targets, p)
    print("LEAKY RELU SCORE")
    print(f"Repeat {i + 1} OOF Log Loss: {loss}")
    scores.append(loss)
    mean_oof_preds.loc[:,target_cols] += p
mean_oof_preds.loc[:, target_cols] /= len(oof_preds)
print(f"Mean OOF Log Loss: {multi_log_loss(train_targets, mean_oof_preds)}")

LEAKY RELU SCORE
Repeat 1 OOF Log Loss: 0.01639366046410945
LEAKY RELU SCORE
Repeat 2 OOF Log Loss: 0.016361492525219325
LEAKY RELU SCORE
Repeat 3 OOF Log Loss: 0.016327601675774708
Mean OOF Log Loss: 0.015998018015484772


# 5. Submission File

In [35]:
test_ = pd.read_csv('../input/lish-moa/test_features.csv')

In [36]:
sig_id = test_[test_["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)

In [37]:
test_preds = sub.copy()
test_preds[target_cols] = 0
test_preds = test_preds.drop("sig_id", axis = 1)
test_preds["sig_id"] = sig_id
test_preds.dropna(inplace=True)
test_preds.drop("sig_id", axis = 1, inplace=True)
for i, model in enumerate(models):
    if type(model) == tf.python.keras.engine.sequential.Sequential:
        preds = model.predict(test_features.values)
        test_preds.loc[:,target_cols] += clip(preds, p_min, p_max)
    else:
        preds = model.predict(test_features.values)
        preds = 1 / (1 + np.exp(-preds))
        test_preds.loc[:,target_cols] += clip(preds, p_min, p_max)
test_preds.loc[:,target_cols] /= len(models)
test_preds["sig_id"] = sig_id
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.merge(test_features[["sig_id"]], test_preds, on = "sig_id", how = "left")
submission.fillna(0, inplace = True)
sub = submission.to_csv('submission.csv', index=False)

In [38]:
submission

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001125,0.001701,0.002070,0.016167,0.022312,0.004645,0.002231,0.005575,0.001000,...,0.001247,0.001885,0.003652,0.001283,0.001053,0.001218,0.001086,0.002906,0.003658,0.002009
1,id_001897cda,0.001008,0.001246,0.002026,0.002671,0.002646,0.002649,0.004246,0.012346,0.005370,...,0.001210,0.001341,0.003430,0.001001,0.012034,0.001098,0.008330,0.001292,0.004247,0.003805
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.001078,0.001050,0.002206,0.012899,0.016743,0.005119,0.002585,0.004552,0.001000,...,0.001040,0.001214,0.003792,0.023096,0.004543,0.001014,0.001454,0.002057,0.001373,0.003001
4,id_0027f1083,0.002172,0.001752,0.001930,0.019041,0.023429,0.003759,0.006138,0.002411,0.001000,...,0.001171,0.001037,0.003204,0.001915,0.001168,0.001324,0.001122,0.001909,0.001000,0.001558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.001067,0.001316,0.001446,0.003927,0.014576,0.002745,0.001685,0.003232,0.001045,...,0.001032,0.004402,0.002690,0.178268,0.008973,0.001063,0.004183,0.001969,0.001363,0.001511
3978,id_ff925dd0d,0.003531,0.003489,0.001197,0.009008,0.025537,0.005249,0.005611,0.003706,0.001086,...,0.001002,0.001235,0.002566,0.001973,0.002062,0.001309,0.001523,0.001523,0.001026,0.001341
3979,id_ffb710450,0.001660,0.001171,0.001211,0.013707,0.048867,0.006277,0.003892,0.004800,0.001000,...,0.001002,0.001008,0.002053,0.001179,0.001625,0.001000,0.001041,0.001493,0.001020,0.001379
3980,id_ffbb869f2,0.001171,0.001458,0.001177,0.024208,0.023858,0.006165,0.004443,0.003147,0.001163,...,0.001000,0.001004,0.003350,0.001308,0.001467,0.001004,0.001494,0.001702,0.001062,0.003166


In [39]:
test_preds

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,sig_id
0,0.001125,0.001701,0.002070,0.016167,0.022312,0.004645,0.002231,0.005575,0.001000,0.012027,...,0.001885,0.003652,0.001283,0.001053,0.001218,0.001086,0.002906,0.003658,0.002009,id_0004d9e33
1,0.001008,0.001246,0.002026,0.002671,0.002646,0.002649,0.004246,0.012346,0.005370,0.008353,...,0.001341,0.003430,0.001001,0.012034,0.001098,0.008330,0.001292,0.004247,0.003805,id_001897cda
2,0.001078,0.001050,0.002206,0.012899,0.016743,0.005119,0.002585,0.004552,0.001000,0.013895,...,0.001214,0.003792,0.023096,0.004543,0.001014,0.001454,0.002057,0.001373,0.003001,id_00276f245
3,0.002172,0.001752,0.001930,0.019041,0.023429,0.003759,0.006138,0.002411,0.001000,0.012204,...,0.001037,0.003204,0.001915,0.001168,0.001324,0.001122,0.001909,0.001000,0.001558,id_0027f1083
4,0.001137,0.001067,0.001814,0.026640,0.039380,0.006277,0.004986,0.002249,0.001000,0.017432,...,0.001259,0.003226,0.005100,0.001308,0.001018,0.001049,0.002202,0.001000,0.002282,id_006fc47b8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,0.001067,0.001316,0.001446,0.003927,0.014576,0.002745,0.001685,0.003232,0.001045,0.005281,...,0.004402,0.002690,0.178268,0.008973,0.001063,0.004183,0.001969,0.001363,0.001511,id_ff7004b87
3620,0.003531,0.003489,0.001197,0.009008,0.025537,0.005249,0.005611,0.003706,0.001086,0.015425,...,0.001235,0.002566,0.001973,0.002062,0.001309,0.001523,0.001523,0.001026,0.001341,id_ff925dd0d
3621,0.001660,0.001171,0.001211,0.013707,0.048867,0.006277,0.003892,0.004800,0.001000,0.016911,...,0.001008,0.002053,0.001179,0.001625,0.001000,0.001041,0.001493,0.001020,0.001379,id_ffb710450
3622,0.001171,0.001458,0.001177,0.024208,0.023858,0.006165,0.004443,0.003147,0.001163,0.027715,...,0.001004,0.003350,0.001308,0.001467,0.001004,0.001494,0.001702,0.001062,0.003166,id_ffbb869f2


In [40]:
def Diff(list1, list2): 
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1)))) 

Diff (submission.columns, pd.read_csv('../input/lish-moa/sample_submission.csv').columns)

[]